In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.graphics.tsaplots import plot_acf

# Read in the data
df = pd.read_csv('cleaned_data.csv')
df

df.loc[df['id'] == 'AS14.01']

,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities,MA
0,2014-03-21,AS14.01,6.200000,0.200000,0.200000,0.134050,17978.907000,6.0,0.0,3139.218,...,49.544,0.000,172.206,239.751,4508.500,915.445,0.000,0.000,598.754,0.134050
1,2014-03-22,AS14.01,6.400000,0.600000,0.400000,0.236880,6142.161000,3.0,1.0,731.429,...,21.076,0.000,0.000,98.143,439.632,37.305,0.000,0.000,117.621,0.190607
2,2014-03-23,AS14.01,6.800000,0.200000,0.800000,0.142741,6773.832001,0.0,0.0,1286.246,...,43.403,0.000,0.000,72.823,900.839,0.000,0.000,30.386,30.086,0.171365
3,2014-03-24,AS14.01,6.000000,0.800000,0.000000,0.078961,15047.351001,10.0,0.0,866.956,...,34.106,0.000,3.010,66.558,3223.626,419.805,0.000,0.000,178.732,0.140922
4,2014-03-25,AS14.01,6.750000,0.500000,0.500000,0.098374,21475.354999,0.0,1.0,1032.768,...,43.054,0.000,0.000,178.819,1919.471,0.000,235.223,0.000,222.893,0.128708
5,2014-03-26,AS14.01,6.600000,-0.200000,0.600000,0.101308,16423.801000,0.0,0.0,1167.497,...,52.331,0.000,0.000,97.498,4592.059,0.000,0.000,0.000,33.365,0.121591
6,2014-03-27,AS14.01,7.000000,0.200000,0.800000,0.159511,17442.149999,2.0,1.0,1229.327,...,42.219,0.000,182.451,58.532,935.381,47.314,0.000,0.000,179.029,0.130728
7,2014-03-28,AS14.01,6.400000,-0.600000,0.600000,0.095698,4923.489000,5.0,0.0,10062.595,...,89.166,233.036,0.000,225.951,512.741,1133.009,0.000,0.000,301.717,0.122759
8,2014-03-29,AS14.01,8.000000,0.200000,1.000000,0.068203,8322.622000,4.0,1.0,1952.630,...,0.000,0.000,0.000,169.594,472.888,52.435,0.000,0.000,600.637,0.110889
9,2014-03-30,AS14.01,7.500000,-0.500000,0.750000,0.049093,4523.214001,0.0,0.0,414.365,...,0.000,0.000,0.000,74.003,167.685,0.000,66.477,0.000,38.296,0.097909


In [12]:
# Convert 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'])

# Create a new column 'period' with period numbers (every period number corresponds to 5 days)
df['period'] = ((df['date'].dt.day - 1) // 5) + 1

agg_dict = {'mood': 'last', 'circumplex.arousal': 'mean', 'circumplex.valence': 'mean', 'activity': 'mean', 'screen': 'sum', 'call': 'sum', 'sms': 'sum', 'appCat.builtin': 'sum', 'appCat.communication': 'sum', 'appCat.entertainment': 'sum', 'appCat.finance': 'sum', 'appCat.game': 'sum', 'appCat.office': 'sum', 'appCat.other': 'sum', 'appCat.social': 'sum', 'appCat.travel': 'sum', 'appCat.unknown': 'sum', 'appCat.weather': 'sum', 'appCat.utilities': 'sum'}

df = df.groupby(['period', 'id']).agg(agg_dict).reset_index()

# Set 'id' as the index
df.set_index('id', inplace=True)


,period,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
id,,,,,,,,,,,,,,,,,,,,
AS14.01,1,7.709987,-0.256775,0.619418,0.084547,122123.530002,18.0,6.0,30059.940,59686.381,10121.864,3559.002,234.741,210.967,1287.534,20488.096,2123.520,30.102,0.000,1960.015
AS14.02,1,7.400000,-0.450000,0.280000,0.290555,33717.162002,20.0,2.0,9102.299,12979.559,3251.429,0.000,0.000,0.000,621.038,1031.374,1051.565,0.000,0.000,0.000
AS14.03,1,7.250000,-0.270000,0.985000,0.103830,40077.018998,33.0,5.0,24719.628,8380.179,24153.050,299.934,0.000,255.004,2492.100,0.000,438.745,532.852,0.000,87.794
AS14.05,1,6.333333,-0.223333,0.960000,0.047640,49807.738996,7.0,12.0,7588.598,32854.000,3233.201,0.000,0.000,0.000,1734.566,3725.636,0.000,1030.332,0.000,129.210
AS14.06,1,7.000000,-0.421667,0.561667,0.202115,58892.346998,17.0,0.0,5456.738,26064.630,8751.175,0.000,2512.675,301.746,616.856,20371.479,1459.782,93.997,7.101,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AS14.24,7,7.000000,1.000000,1.000000,0.212520,1705.078000,0.0,0.0,1063.115,624.496,0.000,0.000,0.000,0.000,7.019,0.000,0.000,0.000,0.000,0.000
AS14.26,7,7.000000,0.600000,0.400000,NaN,0.000000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
AS14.28,7,6.500000,0.000000,0.000000,0.024151,1870.437000,0.0,0.0,818.388,93.215,181.001,159.497,0.000,0.000,40.548,691.166,0.000,0.000,0.000,6.177


In [18]:
df.loc['AS14.01']

,period,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
id,,,,,,,,,,,,,,,,,,,,
AS14.01,1,7.709987,-0.256775,0.619418,0.084547,122123.530002,18.0,6.0,30059.940,59686.381,10121.864,3559.002,234.741,210.967,1287.534,20488.096,2123.520,30.102,0.000,1960.015
AS14.01,2,7.400000,-0.570000,0.510000,0.077951,55269.223002,17.0,0.0,8722.700,31246.798,6900.477,65.259,115.465,40.302,953.555,8453.066,60.247,0.000,0.000,1243.249
AS14.01,3,6.400000,-0.880000,0.790000,0.068051,55854.076998,9.0,2.0,6676.829,23738.985,11622.090,278.872,0.000,0.000,480.205,8245.936,76.732,0.000,0.000,1832.847
AS14.01,4,7.250000,-0.230000,0.700000,0.088262,62668.450000,6.0,3.0,5179.988,30082.996,2540.095,225.398,29.084,0.000,208.020,8648.334,192.764,0.000,0.000,788.816
AS14.01,5,7.200000,0.190000,0.670000,0.109093,137336.626998,27.0,3.0,16126.529,75293.516,8879.434,511.961,0.000,175.216,920.481,18286.416,1555.333,235.223,30.386,3093.298
AS14.01,6,7.800000,-0.250000,0.800000,0.099203,100674.583997,17.0,6.0,24429.124,37893.285,10969.937,1051.258,566.341,182.451,2027.306,12001.185,2018.311,69.625,0.000,2432.069
AS14.01,7,7.400000,0.000000,0.600000,0.070505,11836.834000,6.0,0.0,2961.405,5332.637,1758.920,41.255,51.176,0.000,268.554,1350.655,183.381,29.290,0.000,210.305
